# Memory

<div style="display: flex; justify-content: flex-start; gap: 10px;">
  <img src="./assets/LC_Memory_after.png" style="width:300px; border:1px solid #ccc; border-radius:6px;">
</div>

Persisting messages, or 'agent state' between invocations of the agent.

## Setup

Load and/or check for needed environmental variables

In [2]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env(".env")

# Declaring default model
default_model = "google_genai:gemini-2.5-flash"

LANGSMITH_API_KEY=****3d99
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ials
LANGSMITH_ENDPOINT=****.com
GOOGLE_API_KEY=****zZa4
DEFAULT_MODEL=****lash


In [3]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [4]:
from dataclasses import dataclass

@dataclass
class RuntimeContext:
    db: SQLDatabase

In [5]:
from langchain_core.tools import tool
from langgraph.runtime import get_runtime

@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    runtime = get_runtime(RuntimeContext)
    db = runtime.context.db

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [6]:
SYSTEM_PROMPT = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
"""

In [7]:
from langchain.agents import create_agent

agent = create_agent(
    model=default_model,
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
)

## Repeated Queries

In [8]:
question = "This is Frank Harris, What was the total on my last invoice?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
    context=RuntimeContext(db=db),
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

This is Frank Harris, What was the total on my last invoice?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (6a105934-500b-46bc-bb36-f24c0816f753)
 Call ID: 6a105934-500b-46bc-bb36-f24c0816f753
  Args:
    query: SELECT CustomerId FROM customers WHERE FirstName = 'Frank' AND LastName = 'Harris'
================================= Tool Message =================================
Name: execute_sql

Error: (sqlite3.OperationalError) no such table: customers
[SQL: SELECT CustomerId FROM customers WHERE FirstName = 'Frank' AND LastName = 'Harris']
(Background on this error at: https://sqlalche.me/e/20/e3q8)
================================== Ai Message ==================================
Tool Calls:
  execute_sql (6974e32d-46c1-401a-af8a-85f57e2b5557)
 Call ID: 6974e32d-46c1-401a-af8a-85f57e2b5557
  Args:
    query: SELECT name FROM sqlite_master WHERE typ

In [9]:
question = "What were the titles?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What were the titles?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (ce604e5c-9cc1-409b-8269-0a95a04133a7)
 Call ID: ce604e5c-9cc1-409b-8269-0a95a04133a7
  Args:
    query: SELECT title FROM tracks LIMIT 5
================================= Tool Message =================================
Name: execute_sql

Error: (sqlite3.OperationalError) no such table: tracks
[SQL: SELECT title FROM tracks LIMIT 5]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
================================== Ai Message ==================================
Tool Calls:
  execute_sql (23052ae8-90ff-4466-b867-f5195db3d9a0)
 Call ID: 23052ae8-90ff-4466-b867-f5195db3d9a0
  Args:
    query: SELECT name FROM sqlite_master WHERE type='table';
================================= Tool Message =================================
Name: execute_sql

[('Album',), ('Artist',), ('Cus

## Add memory

In [11]:
from langgraph.checkpoint.memory import InMemorySaver

In [12]:
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage

agent = create_agent(
    model=default_model,
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
    checkpointer=InMemorySaver(),
)

In [13]:
question = "This is Frank Harris, What was the total on my last invoice?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

This is Frank Harris, What was the total on my last invoice?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (29aea950-50a7-409c-877c-dceb7b8238a2)
 Call ID: 29aea950-50a7-409c-877c-dceb7b8238a2
  Args:
    query: SELECT CustomerId FROM Customer WHERE FirstName = 'Frank' AND LastName = 'Harris'
================================= Tool Message =================================
Name: execute_sql

[(16,)]
================================== Ai Message ==================================
Tool Calls:
  execute_sql (b92edde3-38bb-46af-be2b-66aa86511a57)
 Call ID: b92edde3-38bb-46af-be2b-66aa86511a57
  Args:
    query: SELECT InvoiceId, Total FROM Invoice WHERE CustomerId = 16 ORDER BY InvoiceDate DESC LIMIT 1
================================= Tool Message =================================
Name: execute_sql

[(374, 5.94)]
================================== A

In [14]:
question = "What were the titles?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

What were the titles?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (61200e76-889b-49f2-9808-0c7421123ec5)
 Call ID: 61200e76-889b-49f2-9808-0c7421123ec5
  Args:
    query: SELECT T.Name FROM InvoiceLine AS IL JOIN Track AS T ON IL.TrackId = T.TrackId WHERE IL.InvoiceId = 374
================================= Tool Message =================================
Name: execute_sql

[('Holier Than Thou',), ('Through The Never',), ('My Friend Of Misery',), ('The Wait',), ('Blitzkrieg',), ('So What',)]
================================== Ai Message ==================================

The titles on your last invoice were: Holier Than Thou, Through The Never, My Friend Of Misery, The Wait, Blitzkrieg, and So What.


## Try your own queries
Now that there is memory, check the agents recall!

In [16]:
question = "How many does my invoices cost?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

How many does my invoices cost?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (57f15f11-badf-46aa-a275-f4ecce942b49)
 Call ID: 57f15f11-badf-46aa-a275-f4ecce942b49
  Args:
    query: SELECT CustomerId FROM Customer WHERE FirstName = 'Frank' AND LastName = 'Harris'
================================= Tool Message =================================
Name: execute_sql

[(16,)]
================================== Ai Message ==================================
Tool Calls:
  execute_sql (35eda631-497b-4507-b0c4-95d6f6cc7956)
 Call ID: 35eda631-497b-4507-b0c4-95d6f6cc7956
  Args:
    query: SELECT SUM(Total) FROM Invoice WHERE CustomerId = 16
================================= Tool Message =================================
Name: execute_sql

[(37.62,)]
================================== Ai Message ==================================

Your invoices cost a total

In [17]:
question = "How many time have I bought ?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

How many time have I bought ?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (dab06d76-f822-4c0b-841d-df38cf819724)
 Call ID: dab06d76-f822-4c0b-841d-df38cf819724
  Args:
    query: SELECT CustomerId FROM Customer WHERE FirstName = 'Frank' AND LastName = 'Harris'
================================= Tool Message =================================
Name: execute_sql

[(16,)]
================================== Ai Message ==================================
Tool Calls:
  execute_sql (ec91eac6-1628-4a50-9221-6e74fc4a34a3)
 Call ID: ec91eac6-1628-4a50-9221-6e74fc4a34a3
  Args:
    query: SELECT COUNT(InvoiceId) FROM Invoice WHERE CustomerId = 16
================================= Tool Message =================================
Name: execute_sql

[(7,)]
================================== Ai Message ==================================

You have bought 7 times.
